<img src="../img/pandora2d_logo.png" width="500" height="500">

# The purpose of this notebook is to explain the concept of margins

Margins are used when using ROI, see ROI documentation [here](https://pandora2d.readthedocs.io/en/stable/userguide/roi.html).  
These margins allow us to have all the pixels we need to apply treatments to the ROI.

**Notebook plan**  
In the first part, you'll find the theoretical part with diagrams.  
In the second, you'll find a configuration that you can modify, giving you at the end the pixel area required for the input ROI.

## Theoretical part

#### explain image

**Option n°1 : Dmin < 0 & m >= 0 & |Dmin| >= m**

<img src="../img/margin_cas_dmin_negative_and_superior_m.svg" width="900" style="display: block; margin: 0 auto">

**Option n°2 : Dmin > 0 & m >= 0 & Dmin >= m**

<img src="../img/margin_cas_dmin_positive_and_superior_m.svg" width="900" style="display: block; margin: 0 auto">

**Option n°3 : Dmin < 0 & m >= 0 & |Dmin| < m**

<img src="../img/margin_cas_dmin_negative_and_inferior_m.svg" width="900" style="display: block; margin: 0 auto">

**Option n°4 : Dmin > 0 & m >= 0 & Dmin < m**

<img src="../img/margin_cas_dmin_positive_and_inferior_m.svg" width="900" style="display: block; margin: 0 auto">

**The formula which summarises the 4 options is as follows**  
<p style="text-align:center;">$max(m - disp_{min}, 0)$</p>

## Pratical part

### Imports and external functions

In [ ]:
import io
from pathlib import Path
from IPython.display import Image, display
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle 

### Imports of pandora2d

In [ ]:
# Load pandora2d imports
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import create_datasets_from_inputs, get_roi_processing

### Load and visualize input data 

#### Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "../data/left.tif"
img_right_path = "../data/right.tif"

#### Provide output directory to write results

In [ ]:
output_dir = Path.cwd() / "output_margins"
# If necessary, create output dir
output_dir.mkdir(exist_ok=True,parents=True)

#### Mode ROI

In [ ]:
roi_mode = True

#### User configuration

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
        },
        "col_disparity": {"init": 0, "range": 2},
        "row_disparity": {"init": 0, "range": 2},
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -2
        },
        "refinement":{
          "refinement_method": "dichotomy",
          "filter": {"method": "bicubic"},
          "iterations" : 3
        },
    },
    "output": {
        "path": "outputs/test_margins"
    },
}

In [ ]:
user_cfg_with_roi = {
    "input": {
        "left": {
            "img": img_left_path,
            "nodata": "NaN",
        },
        "right": {
            "img": img_right_path,
        },
        "col_disparity": {"init": 0, "range": 1},
        "row_disparity": {"init": 0, "range": 1},
    },
    "ROI":{
        "col": {"first": 9, "last": 10},
        "row": {"first": 5, "last": 9}
    },
    "pipeline":{
        "matching_cost" : {
            "matching_cost_method": "zncc",
            "window_size": 5,
        },
        "disparity": {
            "disparity_method": "wta",
            "invalid_disparity": -2
        },
        "refinement":
        {
          "refinement_method": "dichotomy",
          "filter": {"method": "bicubic"},
          "iterations" : 3
        },
    },
    "output": {
        "path": "outputs/test_margins_with_roi"
    },
}

### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

### Check the configuration and sequence of steps

In [ ]:
if not roi_mode:
    checked_cfg = check_conf(user_cfg, pandora2d_machine)
else:
    # Mode ROI
    checked_cfg = check_conf(user_cfg_with_roi, pandora2d_machine)
pprint(checked_cfg['pipeline'])

#### Create dataset

In [ ]:
if not roi_mode:
    img_left, img_right = create_datasets_from_inputs(input_config=checked_cfg["input"])
else:
    # Mode ROI
    checked_cfg["ROI"]["margins"] = pandora2d_machine.margins_img.global_margins.astuple()
    roi = get_roi_processing(checked_cfg["ROI"], checked_cfg["input"]["col_disparity"], checked_cfg["input"]["row_disparity"])
    print(f'{roi=}')
    img_left, img_right = create_datasets_from_inputs(input_config=checked_cfg["input"], roi=roi)

#### Check margins for dataset

In [ ]:
pandora2d_machine.margins_img.global_margins.astuple()

#### Display masks (roi, disparity, window_size)

In [ ]:
size = 14

In [ ]:
def create_roi_patch(roi, col_disparity, row_disparity, image_margin, size=15):
    upper_left_corner = (roi["col"]["first"], roi["row"]["first"])
    width = roi["col"]["last"] - roi["col"]["first"] + 1
    height = roi["row"]["last"] - roi["row"]["first"] + 1
    return Rectangle(upper_left_corner, width, height, color="red", label="User roi")

In [ ]:
def create_disparity_patch(roi, col_disparity, row_disparity, image_margin, size=15):
    dmin_row = int(np.min(row_disparity["init"])) - row_disparity["range"]
    dmax_row = int(np.max(row_disparity["init"])) + row_disparity["range"]
    dmin_col = int(np.min(col_disparity["init"])) - col_disparity["range"]
    dmax_col = int(np.max(col_disparity["init"])) + col_disparity["range"]
    upper_left_corner = (roi["col"]["first"] + dmin_col, roi["row"]["first"] + dmin_row)
    width = (roi["col"]["last"] + dmax_col) - (roi["col"]["first"] + dmin_col) + 1
    height = (roi["row"]["last"] + dmax_row) - (roi["row"]["first"] + dmin_row) + 1
    return Rectangle(upper_left_corner, width, height, color="green", label="User roi with disparity")

In [ ]:
def create_disparity_with_matchingcost_window_patch(roi, col_disparity, row_disparity, image_margin, size=15):
    dmin_row = int(np.min(row_disparity["init"])) - row_disparity["range"]
    dmax_row = int(np.max(row_disparity["init"])) + row_disparity["range"]
    dmin_col = int(np.min(col_disparity["init"])) - col_disparity["range"]
    dmax_col = int(np.max(col_disparity["init"])) + col_disparity["range"]
    first_row = max(roi["row"]["first"] + dmin_row - image_margin,0)
    last_row = min(roi["row"]["last"] + dmax_row + image_margin, size)
    first_col = max(roi["col"]["first"] + dmin_col - image_margin,0)
    last_col = min(roi["col"]["last"] + dmax_col + image_margin, size)
    upper_left_corner = (first_col, first_row)
    width = (last_col - first_col) + 1
    height = (last_row - first_row) + 1
    return Rectangle(upper_left_corner, width, height, color="blue", label="User roi with disparity + matching_cost window")

In [ ]:
# Left image
left_image = np.full((size,size,3), 0)

roi_row = [roi["row"]["first"], roi["row"]["last"]+1]
roi_col = [roi["col"]["first"], roi["col"]["last"]+1]
left_image[roi_row[0]:roi_row[1], roi_col[0]:roi_col[1], 0] = 255

In [ ]:
# Params
cfg_roi = checked_cfg["ROI"]
col_disparity = checked_cfg["input"]["col_disparity"]
row_disparity = checked_cfg["input"]["row_disparity"]
image_margin = pandora2d_machine.margins_img.global_margins.astuple()[0]

patches_arguments = cfg_roi, col_disparity, row_disparity, image_margin, size
# Get patches
left_image_patches = [
    create_roi_patch(*patches_arguments),
]
right_image_patches = [
    create_disparity_with_matchingcost_window_patch(*patches_arguments),
    create_disparity_patch(*patches_arguments),
    create_roi_patch(*patches_arguments),
]

# Plot figures
fig = plt.figure(figsize=(10,10))
ax0 = fig.add_subplot(1,2,1)
ax0.set_xlim([0,size])
ax0.set_ylim([0,size])
ax0.yaxis.set_inverted(True)
ax0.tick_params(bottom=False, top=True, labelbottom=False,labeltop=True)
for artist in left_image_patches:
    ax0.add_artist(artist)
ax0.set_aspect('equal', 'box')
ax0.grid(True, which='both', linestyle='--', axis='both')
plt.title("Left image")

ax1 = fig.add_subplot(1,2,2)
ax1.set_xlim([0,size])
ax1.set_ylim([0,size])
ax1.yaxis.set_inverted(True)
for artist in right_image_patches:
    ax1.add_artist(artist)
ax1.set_aspect('equal', 'box')
ax1.grid(True, which='both', linestyle='--', axis='both')
plt.title("Right image")

plt.legend(handles=right_image_patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
